In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
import random

from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing import text, sequence
from keras.layers import Embedding, SpatialDropout1D
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input
from keras.optimizers import RMSprop
import keras.backend as K
import keras
from keras.layers import Dense, Input, GRU, LSTM, Bidirectional, Dropout, CuDNNLSTM, CuDNNGRU, GlobalAveragePooling1D, GlobalMaxPool1D, TimeDistributed
from keras.optimizers import RMSprop, Adam
from keras.regularizers import l2
from keras.layers import BatchNormalization, SpatialDropout1D, LSTM
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.engine.topology import Layer, InputSpec
from keras import initializers as initializers, regularizers, constraints

from numpy.random import seed
from tensorflow import set_random_seed
import random as rn
import os
from nltk import tokenize 
import nltk
nltk.download('punkt')

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#2. Get the file
data_path          = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path         = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'


#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/train_cleaned_sent_punkt.csv') 
test_labelled = pd.read_csv(f'{data_path}/test_labelled_cleaned_sent_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_sent_punkt.csv') 

In [0]:
train['mal']         = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [0]:
#X = train.comment_text
#y = train.mal

max_features = 40000
maxlen       = 400
dropout_rate = 0.25
rs           = 42
epochs       = 4
batch_size   = 256
embed_dim    = 50
rec_units    = 150


max_sen_len = 100
max_sent_amount = 4

seed(rs)
set_random_seed(rs)
rn.seed(rs)

os.environ['PYTHONHASHSEED']=str(rs)

In [0]:
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [0]:
def make_hat(MAX_SENT_LENGTH = max_sen_len, MAX_SENTS=max_sent_amount, MAX_NB_WORDS=max_features, EMBEDDING_DIM=embed_dim, rec_units=rec_units):

    sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    embedded_sequences = Embedding(MAX_NB_WORDS+1, EMBEDDING_DIM, trainable=True)(sentence_input)
    embedded_sequences = SpatialDropout1D(dropout_rate)(embedded_sequences)
    l_lstm = Bidirectional(CuDNNGRU(rec_units, return_sequences=True))(embedded_sequences)
    l_att = AttentionWithContext()(l_lstm)
    sentEncoder = Model(sentence_input, l_att)

    review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
    review_encoder = TimeDistributed(sentEncoder)(review_input)
    l_lstm_sent = Bidirectional(CuDNNGRU(rec_units, return_sequences=True))(review_encoder)
    l_att_sent = AttentionWithContext()(l_lstm_sent)

    preds = Dense(1, activation='sigmoid')(l_att_sent)
    model = Model(review_input, preds)

    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(clipvalue=1, clipnorm=1),
                  metrics=['acc'])
    return model

In [0]:
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c = 0

In [0]:
def clean_str(string):
    #string = string.replace(",", ".").replace(";", ".").replace(":", ".").replace("-", ".")
    return string.strip().lower()

def tok_sentence(s):
    temp = tokenizer.texts_to_sequences(s)
    if len(temp)==0:
        return np.array([0])
    return temp

In [0]:
for c, (train_index, val_index) in enumerate(kf.split(X, y)):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
    
    
    train_posts = []
    train_labels = []
    train_texts = []
    
    tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(X_train)
    
    #TRAIN
    for i, value in enumerate(X_train):
        if(i%10000==0):
            print(i)
        text = clean_str(value)
        train_texts.append(text)
        sentences = tokenize.sent_tokenize(text)
        sentences = tok_sentence(sentences)
        x = len(sentences)<max_sent_amount
        while x:
            sentences.append(np.array([0])) 
            x = len(sentences)<max_sent_amount

        if len(sentences)>max_sent_amount:
            sentences = sentences[0:max_sent_amount]
        sentences = sequence.pad_sequences(sentences, maxlen=max_sen_len)

        train_posts.append(sentences)
    
    val_posts = []
    val_labels = []
    val_texts = []

    #VAL
    for i, value in enumerate(X_val):
        if(i%10000==0):
            print(i)
        text = clean_str(value)
        val_texts.append(text)
        sentences = tokenize.sent_tokenize(text)
        sentences = tok_sentence(sentences)


        x = len(sentences)<max_sent_amount
        while x:
            sentences.append(np.array([0])) 
            x = len(sentences)<max_sent_amount

        if len(sentences)>max_sent_amount:
            sentences = sentences[0:max_sent_amount]
        sentences = sequence.pad_sequences(sentences, maxlen=max_sen_len)
        val_posts.append(sentences)
    
    X_train = np.array(train_posts)
    y_train = np.array(y_train)
    X_val =  np.array(val_posts)
    y_val = np.array(y_val)
    
    del train_posts
    del val_posts
    
    
    model                = make_hat()
    print('Fitting')
    history              = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_val, y_val), epochs=4, shuffle=True, verbose=1)
    probs                = model.predict(X_val, batch_size=batch_size, verbose=1)
    
    model.save_weights(f'{cv_models_path}/HAN_fold_{c}.h5')
    
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {round(auc_f,3)}')
    print(f' roc auc {round(roc_f,3)}')
    del model
    K.clear_session()

 fold 0
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000


W0729 20:54:12.872534 139624020133760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0729 20:54:12.873977 139624020133760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0729 20:54:12.882664 139624020133760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0729 20:54:12.903214 139624020133760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0729 20:54:12.912525 

Fitting
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 75s 594us/step - loss: 0.1832 - acc: 0.9373 - val_loss: 0.1204 - val_acc: 0.9579
Epoch 2/4
126974/126974 [==============================] - 72s 568us/step - loss: 0.1265 - acc: 0.9526 - val_loss: 0.1156 - val_acc: 0.9597
Epoch 3/4
126974/126974 [==============================] - 72s 564us/step - loss: 0.1156 - acc: 0.9565 - val_loss: 0.1148 - val_acc: 0.9613
Epoch 4/4
31745/31745 [==============================] - 5s 166us/step
 average precision 0.871
 roc auc 0.966
 fold 1
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
Fitting
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 73s 578us/step - loss: 0.1956 - acc: 0.9322 - val_loss: 0.1193 - val_acc: 0.9581
Epoch 2/4
126974/126974 [==============================] - 72s 564us/step - loss: 0.1261 - acc: 0.9530 - v

In [0]:
round(np.array(auc).mean(), 3)

0.839

In [0]:
round(np.array(roc).mean(),3)

0.963

In [0]:
X_train   = X
y_train   = y
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_features, oov_token='unknown')
tokenizer.fit_on_texts(X_train)

train_posts = []
train_labels = []
train_texts = []

# FULL TRAIN
for i, value in enumerate(X):
    if(i%10000==0):
        print(i)
    text = clean_str(value)
    train_texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentences = tok_sentence(sentences)
    x = len(sentences)<max_sent_amount
    while x:
        sentences.append(np.array([0])) 
        x = len(sentences)<max_sent_amount

    if len(sentences)>max_sent_amount:
        sentences = sentences[0:max_sent_amount]
    sentences = sequence.pad_sequences(sentences, maxlen=max_sen_len)

    train_posts.append(sentences)

    
test_posts = []
test_labels = []
test_texts = []
    
    
#Test
for i, value in enumerate(X_test):
    if(i%10000==0):
        print(i)
    text = clean_str(value)
    test_texts.append(text)
    sentences = tokenize.sent_tokenize(text)
    sentences = tok_sentence(sentences)
    x = len(sentences)<max_sent_amount
    while x:
        sentences.append(np.array([0])) 
        x = len(sentences)<max_sent_amount

    if len(sentences)>max_sent_amount:
        sentences = sentences[0:max_sent_amount]
    sentences = sequence.pad_sequences(sentences, maxlen=max_sen_len)

    test_posts.append(sentences)
    
    
X_train = np.array(train_posts)
y_train = np.array(y)
X_test =  np.array(test_posts)
y_test = np.array(y_test)

del train_posts
del test_posts

model   = make_hat()

y_train = np.array(y_train)
y_test  = np.array(y_test)

print('Fitting')
model.fit(X_train, y_train,   batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)
probs = model.predict(X_test, batch_size=batch_size, verbose=1)
auc_f = average_precision_score(y_test, probs)
roc_f = roc_auc_score(y_test, probs)
model.save_weights(f'{models_path}/HAN.h5')
del model

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
0
10000
20000
30000
40000
50000
60000
Fitting
Epoch 1/4
158719/158719 [==============================] - 84s 530us/step - loss: 0.1749 - acc: 0.9405
Epoch 2/4
158719/158719 [==============================] - 83s 522us/step - loss: 0.1219 - acc: 0.9550
Epoch 3/4
158719/158719 [==============================] - 83s 522us/step - loss: 0.1123 - acc: 0.9579
Epoch 4/4
64830/64830 [==============================] - 11s 162us/step


In [0]:
round(auc_f,3)

0.833

In [0]:
round(roc_f,3)

0.964